# Coles

In [ ]:
using DataFrames
using Extremes
using Distributions
using Gadfly

### Model validation functions

#### Non-stationary standardization

In [ ]:
function standardize(eva::Extremes.MaximumLikelihoodEVA)
    if  eva.model isa Extremes.BlockMaxima
        return standardize.(Extremes.getdistribution(eva), eva.model.data[eva.model.dataid])
    elseif eva.model isa Extremes.PeaksOverThreshold
        return standardize.(Extremes.getdistribution(eva), eva.model.data[eva.model.dataid], Extremes.model.threshold)
    end
end

function standardize(dist::GeneralizedExtremeValue, data::Real)
    return (1 / dist.ξ) * log(1 + dist.ξ * (data - dist.μ) / dist.σ)
end

function standardize(dist::GeneralizedPareto, data::Real, threshold::Real)
   return # TODO
end

function standarddist(test::Type)
    if test == GeneralizedExtremeValue
        return Gumbel()
    elseif test == GeneralizedPareto
        return Exponential()
    end
end

#### Probability plot

In [ ]:
function probabilityplot(eva::Extremes.MaximumLikelihoodEVA)
    z = sort(eva.model.data[eva.model.dataid])
    m = length(z)
    dist = Extremes.getdistribution(eva)
    
    return probabilityplot(z, m, dist)
end

function residualprobabilityplot(eva::Extremes.MaximumLikelihoodEVA)
    std = standardize(eva)
    
    z = sort(std)
    m = length(z)
    
    return probabilityplot(z, m, standarddist(eva.model.distribution), title = "Residual Probability Plot")
end
    

function probabilityplot(z::Array{<:Real}, m::Integer, dist::Distribution; title::String = "Probability Plot")
    G̃ = collect(1.0:m) ./ (m + 1)
    Ĝ = cdf.(dist, z)

    l1 = layer(x = G̃, y = Ĝ, Geom.point, Theme(default_color="green"))
    l2 = layer(x = 0.0:0.5:1.0, y = 0.0:0.5:1.0, Geom.line)
    
    return plot(l1, l2, Guide.xlabel("Empirical"), Guide.ylabel("Model"), Guide.title(title))
end

#### Quantile plot

In [ ]:
function quantileplot(eva::Extremes.MaximumLikelihoodEVA)
    z = sort(eva.model.data[eva.model.dataid])
    m = length(z)
    dist = Extremes.getdistribution(eva)
    
    return quantileplot(z, m, dist)
end

function residualquantileplot(eva::Extremes.MaximumLikelihoodEVA)
    std = standardize(eva)
    
    z = sort(std)
    m = length(z)
    
    return quantileplot(z, m, standarddist(eva.model.distribution), title = "Residual Quantile Plot")
end

function quantileplot(z::Array{<:Real}, m::Integer, dist::Distribution; title::String = "Quantile Plot")
    Ĝ⁻¹ = quantile.(dist, collect(1.0:m) / (m + 1))

    l1 = layer(x = Ĝ⁻¹, y = z, Geom.point, Theme(default_color="green"))
    l2 = layer(x = Ĝ⁻¹[1]:(Ĝ⁻¹[m] - Ĝ⁻¹[1])/2:Ĝ⁻¹[m], y = Ĝ⁻¹[1]:(Ĝ⁻¹[m] - Ĝ⁻¹[1])/2:Ĝ⁻¹[m], Geom.line)
    
    return plot(l1, l2, Guide.title(title), Guide.xlabel("Model"), Guide.ylabel("Empirical"),
        Coord.cartesian(xmin = Ĝ⁻¹[1], ymin = z[1]))
end

#### Return level plot

In [ ]:
function returnlevelplot(eva::Extremes.MaximumLikelihoodEVA)
    z = sort(eva.model.data[eva.model.dataid])
    m = length(z)
    dist = Extremes.getdistribution(eva)

    return returnlevelplot(z, m, dist)
end

function returnlevelplot(z::Array{<:Real}, m::Integer, dist::Ty) where Ty<:Distribution
    T = 2:0.2:10
    logT = log.(T)
    p = 1 .- 1 ./ T
    
    q = z[Int.(round.(p .* m))]
    
    layers = []
    push!(layers, layer(x = logT, y = q, Geom.point, Theme(default_color="green")))

    shape = [-0.2, 0.0, 0.2]
    for ξ in shape
        pd = Ty(dist.μ, dist.σ, ξ)
        q = quantile.(pd, p)
        push!(layers, layer(x = logT, y = q, Geom.line))
    end
    
    return plot(layers..., Guide.title("Return Level Plot"), Guide.xlabel("Period"), Guide.ylabel("Level"),
        Coord.cartesian(xmin = logT[1]))
end

#### Density plot

In [ ]:
function densityplot(eva::Extremes.MaximumLikelihoodEVA; nbars::Integer = 10)
    z = sort(eva.model.data[eva.model.dataid])
    m = length(z)
    dist = Extremes.getdistribution(eva)

    return densityplot(z, m, dist)
end

function densityplot(z::Array{<:Real}, m::Integer, dist::Distribution; nbars::Integer = 10)
    start = z[1]
    finish = z[end]
    step = (finish - start) / nbars
    
    function density(v::Real)
        return sum((z .>= (v - step / 2) * ones(m)) .& (z .<= (v + step / 2) * ones(m)))
    end
    
    zb = (start + step/2):step:(finish + step/2)
    db = density.(zb) * 1 / (m * step)
    lb = layer(x = zb, y = db, Geom.BarGeometry)
    
    zl = start:0.01:(finish + step)
    dl = pdf.(dist, zl)
    ll = layer(x = zl, y = dl, Geom.line, Theme(default_color = "Green"))
    
    return plot(ll, lb, Guide.title("Density Plot"), Guide.xlabel("z"), Guide.ylabel("f(z)"))
end

`validationplots` is the function that should be called to display all the graphs 

In [ ]:
function validationplots(eva::Extremes.MaximumLikelihoodEVA; nbars::Integer = 10)
    z = sort(eva.model.data[eva.model.dataid])
    m = length(z)
    dist = Extremes.getdistribution(eva)
    
    probabilityPlot = probabilityplot(z, m, dist)
    quantilePlot = quantileplot(z, m, dist)
    returnLevelPlot = returnlevelplot(z, m, dist)
    densityPlot = densityplot(z, m, dist, nbars = nbars)
    
    return gridstack([probabilityPlot quantilePlot; returnLevelPlot densityPlot])
end

function residualvalidationplots(eva::Extremes.MaximumLikelihoodEVA)
    std = standardize(eva)

    z = sort(std)
    m = length(z)
    dist = standarddist(eva.model.distribution)
    
    probabilityPlot = probabilityplot(z, m, dist, title = "Residual Probability Plot")
    quantilePlot = quantileplot(z, m, dist, title = "Residual Quantile Plot")
    
    return hstack(probabilityPlot, quantilePlot)
end

### Utility function

`printparams` prints the parameter estimation for a block maxima structure

In [ ]:
function printparams(eva::Extremes.MaximumLikelihoodEVA)
    for (index, value) in pairs(eva.model.paramindex)
        println(index, " : ", eva.θ̂[value])
    end
end

## 3.4.1 Annual Maximum Sea-levels at Port Pirie

In [ ]:
raw = load("portpirie")
df = DataFrame(raw)
data = df[:, :SeaLevel];

In [ ]:
gevEVA = gevfit(data)
gev = Extremes.getdistribution(gevEVA)

In [ ]:
validationplots(gevEVA)

10-year return level with 95% accuracy

In [ ]:
year = 10
accuracy = 95
p = 1 - 1 / year

z = Extremes.quantile(gevEVA.model, gevEVA.θ̂, p)
v = sqrt(Extremes.quantilevar(gevEVA, p))

a = 1 - (1 - accuracy/100) / 2

i = quantile(Normal(), a)

println("z = [", z - i * v, ", ", z + i * v, "]")

## 3.4.2 Glass Fiber Strength Example 

In [ ]:
raw = load("glass")
df = DataFrame(raw)
data = -1 * df[:, :Strength]; # minima

In [ ]:
gevEVA = gevfit(data)
gev = Extremes.getdistribution(gevEVA)

In [ ]:
validationplots(gevEVA)

## 4.4.1 Daily Rainfall Data

In [ ]:
raw = load("rain")
df = DataFrame(raw)
data = df[:, :Rainfall]

threshold = 30
exceedances = data[data.>threshold] .- threshold

gpEVA = Extremes.gpfit(exceedances, threshold = [threshold]) # TODO: Export
gp = Extremes.getdistribution(gpEVA)

In [ ]:
validationplots(gpEVA)

100-year return level with 95% accuracy

In [ ]:
year = 100
mperyear = 365
accuracy = 95

k = sum(data .> threshold)
n = length(data)

m = year * mperyear * k / n

p = 1 - 1 / m

z = Extremes.quantile(gpEVA.model, gpEVA.θ̂, p) + threshold
v = sqrt(Extremes.quantilevar(gpEVA, p))

a = 1 - (1 - accuracy/100) / 2

i = quantile(Normal(), a)

println("z = [", z - i * v, ", ", z + i * v, "]")

## 4.4.2 Dow Jones Index Series

In [ ]:
raw = lraw = load("dowjones")
df = DataFrame(raw)
data = df[:, :Index]

function dowjonestransformation(i::Integer)
   return 100 * (log(data[i]) - log(data[i - 1]))
end

i = 2:length(data)
transformed = dowjonestransformation.(i)

threshold = 2
exceedances = transformed[transformed.>threshold] .- threshold

gpEVA = Extremes.gpfit(exceedances, threshold = [threshold]) # TODO : export
gp = Extremes.getdistribution(gpEVA)

In [ ]:
validationplots(gpEVA)

## 6.3.1 Annual Maximum Sea-levels

In [ ]:
quantileχ²₁ = quantile(Chisq(1), 0.95)

### Port Pirie

In [ ]:
raw = load("portpirie")
df = DataFrame(raw)
data = df[:, :SeaLevel]

t = collect(1:length(data))

d = Dict(:data => data, :t => t, :n => length(data));

#### Stationary

In [ ]:
gevEVA = gevfit(data)
printparams(gevEVA)
gev = Extremes.getdistribution(gevEVA)

mlogls = sum(log.(pdf.(gev, data)));

#### μ linear

In [ ]:
cov = Dict(:μ => [:t], :ϕ => Symbol[], :ξ => Symbol[])
EVA = gevfit(d, :data, Covariate = cov)
printparams(EVA)
linμgev = Extremes.getdistribution(EVA);

mloglμlinear = sum(log.(pdf.(linμgev, data)))

D = 2(mloglμlinear - mlogls)

println()
println("linear μ vs. stationary μ")
println(D, " < ", quantileχ²₁)
println("Stationary is a better represantation")

### Fremantle

In [ ]:
raw = load("fremantle")
df = DataFrame(raw)
data = df[:, :SeaLevel]
year = df[:, :Year]
soi = df[:, :SOI]

t = collect(1:length(data))
t2 = t.^2

d = Dict(:data => data, :t => t, :t2 => t2, :soi => soi, :n => length(data));

#### Stationary

In [ ]:
gevEVA = gevfit(data)
printparams(gevEVA)
gev = Extremes.getdistribution(gevEVA)

mloglstationary = sum(log.(pdf.(gev, data)));

#### μ linear

In [ ]:
cov = Dict(:μ => [:t], :ϕ => Symbol[], :ξ => Symbol[])
EVA = gevfit(d, :data, Covariate = cov)
printparams(EVA)
linμgev = Extremes.getdistribution(EVA);

mloglμlinear = sum(log.(pdf.(linμgev, data)))

println()
println("linear μ vs. stationary μ")
D = 2(mloglμlinear - mloglstationary)
println(D, " > ", quantileχ²₁)
println("Linear is a better representation")

In [ ]:
t₁ =  1897

l1 = layer(x = year, y = data)

μs = EVA.θ̂[EVA.model.paramindex[:μ]]
println("μ̂  = ", μs)

β̂₀ = μs[1]
β̂₁ = μs[2]
l = β̂₀ .+ β̂₁ .* (year .- t₁)

l2 = layer(x = year, y = l, Geom.line, Theme(default_color = "green"))

plot(l2, l1, Coord.cartesian(xmin = year[1]),
    Guide.title("Fitted estimates for μ"), Guide.xlabel("Year"), Guide.ylabel("Sea-level"))

In [ ]:
residualvalidationplots(EVA)

####  μ quadratic

In [ ]:
cov = Dict(:μ => [:t, :t2], :ϕ => Symbol[], :ξ => Symbol[])
EVA = gevfit(d, :data, Covariate = cov)
printparams(EVA)
quadμgev = Extremes.getdistribution(EVA);

mloglμquadratic = sum(log.(pdf.(quadμgev, data)))

println()
println("quadratic μ vs. linear μ")
D = 2(mloglμquadratic - mloglμlinear)
println(D, " < ", quantileχ²₁)
println("Linear is a better representation")

#### σ linear

In [ ]:
cov = Dict(:μ => [:t], :ϕ => [:t], :ξ => Symbol[])
EVA = gevfit(d, :data, Covariate = cov)
printparams(EVA)
linσgev = Extremes.getdistribution(EVA);

mloglμlinearσlinear = sum(log.(pdf.(linσgev, data)))

println()
println("linear σ vs. stationary σ")
D = 2(mloglμlinearσlinear - mloglμlinear)
println(D, " < ", quantileχ²₁)
println("Stationary is a better representation")

#### μ SOI

In [ ]:
cov = Dict(:μ => [:soi], :ϕ => Symbol[], :ξ => Symbol[])
EVA = gevfit(d, :data, Covariate = cov)
printparams(EVA)
soiμgev = Extremes.getdistribution(EVA);

mloglSOIμ = sum(log.(pdf.(soiμgev, data)))

println()
println("SOI μ vs. stationary μ")
D = 2(mloglSOIμ - mloglstationary)
println(D, " > ", quantileχ²₁)
println("SOI is a better representation")

#### μ linear + SOI

In [ ]:
cov = Dict(:μ => [:t, :soi], :ϕ => Symbol[], :ξ => Symbol[])
EVA = gevfit(d, :data, Covariate = cov)
printparams(EVA)
linsoiμgev = Extremes.getdistribution(EVA);

mloglSOIlinearμ = sum(log.(pdf.(linsoiμgev, data)))

println()
println("linear + SOI μ vs. linear μ")
D = 2(mloglSOIlinearμ - mloglμlinear)
println(D, " > ", quantileχ²₁)
println("Linear + SOI is a better representation")

## 6.3.4 Daily Rainfall Data 

In [ ]:
raw = load("rain")
df = DataFrame(raw)
data = df[:, :Rainfall];

threshold = 30
exceedances = data[data.>threshold] .- threshold

EVA = Extremes.gpfit(exceedances, threshold = [threshold])
stationarygp = Extremes.getdistribution(EVA)

mloglstationary = sum(log.(pdf.(stationarygp, exceedances)));

In [ ]:
t = collect(1:length(exceedances))

d = Dict(:data => exceedances, :t => t, :n => length(exceedances))

cov = Dict(:μ => [], :ϕ => Symbol[:t], :ξ => Symbol[])
EVA = Extremes.gpfit(d, :data, Covariate = cov) # TODO : Export
printparams(EVA)
linσgp = Extremes.getdistribution(EVA)

mloglσlinear = sum(log.(pdf.(linσgp, exceedances)))

println()
println("linear σ vs. stationary σ")
D = 2(mloglstationary - mloglσlinear)
println(D, " < ", quantileχ²₁)
println("Stationary is a better representation")

## 9.1.3 Example: Port Pirie Annual Maximum Sea-levels 
The priors used by Extremes.jl and by Coles differ, but the results should remain close

In [ ]:
raw = load("portpirie")
df = DataFrame(raw)
data = df[:, :SeaLevel];

EVA = gevfitbayes(data)

In [ ]:
μs = EVA.sim.value[:, EVA.model.paramindex[:μ]]
ϕs = EVA.sim.value[:, EVA.model.paramindex[:ϕ]]
ξs = EVA.sim.value[:, EVA.model.paramindex[:ξ]]
q = quantile(EVA, 0.99)

println("μ̂ = ", mean(μs), " (", sqrt(var(μs)), ")")
println("σ̂ = ", mean(exp.(ϕs)), " (", sqrt(var(exp.(ϕs))), ")")
println("ξ̂ = ", mean(ξs), " (", sqrt(var(ξs)), ")")
println("ẑ = ", mean(q), " (", sqrt(var(q)), ")")